In [1]:
from geoband.API import *
import pandas as pd

import folium
import geopandas as gpd

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data9 = pd.read_csv('data/9.오산시_어린이보호구역.csv')
data9.tail()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
86,어린이집,시립세마어린이집,Y,1,12,127.044916,37.184608
87,어린이집,미래숲어린이집,Y,1,6,127.044384,37.145326
88,어린이집,시립은여울어린이집,N,0,20,127.066860,37.170952
89,어린이집,설리반어린이집,N,0,5,127.061041,37.164127
90,어린이집,세교복지타운어린이집,Y,1,15,127.044835,37.177752


In [3]:
print(data9[data9['CCTV설치여부'] == 'N']['CCTV설치대수'].value_counts())
print(data9[data9['CCTV설치여부'] == 'Y']['CCTV설치대수'].value_counts())

0    16
1     1
Name: CCTV설치대수, dtype: int64
1    51
2    16
3     7
Name: CCTV설치대수, dtype: int64


In [4]:
# 23번 인덱스에 CCTV설치여부 : N, CCTV설치대수 : 1
CCTV_N = data9['CCTV설치여부'] == 'N'
CCTV_COUNT = data9['CCTV설치대수'] != 0

data9[CCTV_N & CCTV_COUNT].index

Int64Index([23], dtype='int64')

In [5]:
# 23번 인덱스 삭제
data9.drop(23, axis = 0).reset_index(drop = True, inplace = True)
data9.tail()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
86,어린이집,시립세마어린이집,Y,1,12,127.044916,37.184608
87,어린이집,미래숲어린이집,Y,1,6,127.044384,37.145326
88,어린이집,시립은여울어린이집,N,0,20,127.066860,37.170952
89,어린이집,설리반어린이집,N,0,5,127.061041,37.164127
90,어린이집,세교복지타운어린이집,Y,1,15,127.044835,37.177752


In [6]:
def make_geom(df, lon, lat):
    geom = gpd.points_from_xy(df[lon], df[lat])
    df = gpd.GeoDataFrame(df, geometry = geom)
    df.crs = {'init' : 'epsg:4326'}
    return df

In [7]:
data9 = make_geom(data9, '보호구역_경도', '보호구역_위도')
data9.tail()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도,geometry
86,어린이집,시립세마어린이집,Y,1,12,127.044916,37.184608,POINT (127.04492 37.18461)
87,어린이집,미래숲어린이집,Y,1,6,127.044384,37.145326,POINT (127.04438 37.14533)
88,어린이집,시립은여울어린이집,N,0,20,127.066860,37.170952,POINT (127.06686 37.17095)
89,어린이집,설리반어린이집,N,0,5,127.061041,37.164127,POINT (127.06104 37.16413)
90,어린이집,세교복지타운어린이집,Y,1,15,127.044835,37.177752,POINT (127.04484 37.17775)


In [8]:
from fiona.crs import from_epsg

def make_geom_circle(df):
    geom_circle_list = []
    for idx in df.index:
        geom = df.geometry.iloc[idx]
        geom_circle = geom.buffer(50)
        geom_circle_list.append(geom_circle)
    df = gpd.GeoDataFrame(df, geometry = geom_circle_list, crs=from_epsg(5181))
    return df

In [9]:
# 미터 단위 좌표계 : EPSG:5186
data9.to_crs(epsg=5181, inplace = True)

In [10]:
data9 = make_geom_circle(data9)
data9.to_crs(epsg=4326, inplace = True)

In [11]:
data9.tail()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도,geometry
86,어린이집,시립세마어린이집,Y,1,12,127.044916,37.184608,"POLYGON ((127.04548 37.18461, 127.04548 37.184..."
87,어린이집,미래숲어린이집,Y,1,6,127.044384,37.145326,"POLYGON ((127.04495 37.14533, 127.04494 37.145..."
88,어린이집,시립은여울어린이집,N,0,20,127.066860,37.170952,"POLYGON ((127.06742 37.17095, 127.06742 37.170..."
89,어린이집,설리반어린이집,N,0,5,127.061041,37.164127,"POLYGON ((127.06160 37.16413, 127.06160 37.164..."
90,어린이집,세교복지타운어린이집,Y,1,15,127.044835,37.177752,"POLYGON ((127.04540 37.17775, 127.04540 37.177..."


In [12]:
data2 = gpd.read_file('new_data/data2_link_notzero.geojson')

In [13]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 12)

folium.Choropleth(geo_data = data2, 
                  data = data2,
                  columns = ('gid', 'accident_cnt'), 
                  key_on='feature.properties.gid',
                  fill_color = 'PuRd', 
                  legend_name='교통사고수').add_to(m)

folium.GeoJson(data9).add_to(m) 
m

In [14]:
data9.to_file('new_data/data9_buffer.geojson', driver='GeoJSON')